In [ ]:
import os
import sys
import json
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from sklearn.metrics import ConfusionMatrixDisplay
from tqdm import tqdm

sys.path.append("..")

DATA_PATH = os.path.join("..", "data")
CHECKPOINTS_PATH =  os.path.join("..", "checkpoints")
device = "cuda"

### Custom models

1. AlexNet


In [34]:
class AlexNet_32x32(nn.Module):
    def __init__(self, num_classes: int = 10, dropout: float = 0.5) -> None:
        super().__init__()
        # Convolution
        self.conv1 = nn.Conv2d(3, 64, kernel_size=5, stride=1, padding=1)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=1)

        self.conv2 = nn.Conv2d(64, 192, kernel_size=3, stride=1, padding=1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=1)

        self.conv3 = nn.Conv2d(192, 256, kernel_size=3, stride=1)
        self.maxpool3 = nn.MaxPool2d(kernel_size=3, stride=2)

        self.conv4 = nn.Conv2d(256, 256, kernel_size=3, stride=1)
        self.maxpool4 = nn.MaxPool2d(kernel_size=3, stride=2)

        self.relu = nn.ReLU(inplace=True)

        # Global average pooling
        self.global_avg_pool = nn.AdaptiveAvgPool2d((4, 4))

        # Classification
        self.dropout1 = nn.Dropout(dropout)
        self.lin1 = nn.Linear(256 * 4 * 4, 4096)

        self.dropout2 = nn.Dropout(dropout)
        self.lin2 = nn.Linear(4096, 4096)

        self.dropout3 = nn.Dropout(dropout)
        self.lin3 = nn.Linear(4096, num_classes)


    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.maxpool1(self.relu(self.conv1(x)))
        x = self.maxpool2(self.relu(self.conv2(x)))
        x = self.maxpool3(self.relu(self.conv3(x)))
        x = self.maxpool4(self.relu(self.conv4(x)))

        x = self.global_avg_pool(x)

        x = torch.flatten(x, 1)

        x = self.lin1(self.dropout1(x))
        x = self.lin2(self.dropout2(x))
        x = self.lin3(self.dropout3(x))

        return x

2. ResNet

In [35]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1, downsampler = None):
        super().__init__()
        self.downsampler = downsampler

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.relu = nn.ReLU(inplace=True)


    def forward(self, x):
        identity = x
        if self.downsampler is not None:
            identity = self.downsampler(identity)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)

        out = out + identity
        out = self.relu(out)
        return out


class ResNet_32x32(nn.Module):
    def __init__(self, num_classes: int = 10, block: nn.Module = ResidualBlock):
        super().__init__()
        self.conv = nn.Conv2d(3, 64, (3, 3), stride=1, padding=1)
        self.bnorm = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)

        self.max_pool1 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.res_layer1 = self._residual_layer(block, 64, 128, 2)

        self.max_pool2 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.res_layer2 = self._residual_layer(block, 128, 256, 2)

        self.max_pool3 = nn.MaxPool2d(kernel_size=3, stride=1)
        self.res_layer3 = self._residual_layer(block, 256, 512, 2)

        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)

        self.lin = nn.Linear(512, num_classes)


    def _residual_layer(self, block, in_channels, out_channels, blocks_num, stride=1):
        """Creates a residual layer consisting out of residual blocks"""
        downsampler = None
        if stride != 1 or in_channels != out_channels:
            downsampler = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

        layers = []
        layers.append(block(in_channels, out_channels, stride, downsampler))
        for _ in range(blocks_num - 1):
            layers.append(block(out_channels, out_channels, stride))

        return nn.Sequential(*layers)


    def forward(self, x):
        x = self.relu(self.bnorm(self.conv(x)))

        x = self.res_layer1(self.max_pool1(x))
        x = self.res_layer2(self.max_pool2(x))
        x = self.res_layer3(self.max_pool3(x))

        x = self.global_avg_pool(x)
        x = torch.flatten(x, 1)
        x = self.lin(x)
        return x

### Data prep

In [2]:
CINIC_MEAN = [0.47889522, 0.47227842, 0.43047404]
CINIC_STD = [0.24205776, 0.23828046, 0.25874835]


def get_dataset(
        path: str,
        batch_size: int,
        shuffle: bool,
        use_augmentations: bool,
) -> DataLoader:
    augmentations = ([
        # TODO: add augmentations
    ] if use_augmentations else [])
    transform = transforms.Compose([*augmentations,
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=CINIC_MEAN,std=CINIC_STD)])

    ds = torchvision.datasets.ImageFolder(path, transform=transform)
    loader = DataLoader(ds, batch_size=batch_size, shuffle=shuffle, num_workers=3)
    return loader


def get_cinic(
        data_path: str,
        batch_size: int = 256
) -> tuple[DataLoader, DataLoader, DataLoader]:
    train_path = os.path.join(data_path, "train")
    valid_path = os.path.join(data_path, "valid")
    test_path = os.path.join(data_path, "test")

    cinic_train = get_dataset(train_path, batch_size, True, True)
    cinic_validation = get_dataset(valid_path, batch_size, False, False)
    cinic_test = get_dataset(test_path, batch_size, False, False)

    return cinic_train, cinic_validation, cinic_test

### Utils

In [3]:
import numpy as np
import torch
import random


def set_seed(seed: int):
    random.seed(seed)
    np.random.RandomState(seed)
    torch.manual_seed(seed)


def get_device():
    return "cuda:0" if torch.cuda.device_count() > 0 else 'cpu'

### Train

In [4]:
MODELS = {
    "AlexNet": AlexNet_32x32,
    "ResNet": ResNet_32x32
}

OPTIMIZERS = {
    "SGD": torch.optim.SGD,
    "AdamW": torch.optim.AdamW
}


def train(config: dict, data_path: str):
    # Set seed for reproducibility
    seed = int(config["seed"]) if "seed" in config else 0
    set_seed(seed)

    device = get_device()
    print("Device: ", device)

    checkpoint = config["checkpoint_folder"]
    os.makedirs(checkpoint, exist_ok=True)

    with open(os.path.join(checkpoint, "config.json"), "w") as f:
        f.write(json.dumps(config))

    batch_size = int(config["batch_size"]) if "batch_size" in config else 256
    cinic_train, cinic_valid, cinic_test = get_cinic(data_path, batch_size=batch_size)
    model: nn.Module = MODELS[config["model"]](**config["model_params"]).to(device)
    optimizer: torch.optim.Optimizer = OPTIMIZERS[config["optimizer"]](params=model.parameters(),
                                                **config["optimizer_params"])
    criterion = nn.CrossEntropyLoss()

    epochs = int(config["epochs"])
    metrics = {"loss": [float("+inf")], "accuracy": [0.0], "val_loss": [float("+inf")], "val_accuracy": [0.0]}
    best_model = model
    warmup_epochs = int(config["warmup_epochs"])

    best_loss = float('inf')
    best_loss_epoch = -1

    classes = cinic_test.dataset.classes
    classes = sorted(classes, key=lambda x: cinic_test.dataset.class_to_idx[x])

    # Training
    for epoch in range(epochs):
        print("Epoch", epoch)

        print("Processing training")
        model.train()
        accuracy, loss = train_epoch(model, cinic_train, optimizer, criterion)
        metrics["accuracy"].append(accuracy)
        metrics["loss"].append(loss)

        print("Processing validation")
        model.eval()
        val_accuracy, val_loss = valid_epoch(model, cinic_valid, criterion)
        metrics["val_accuracy"].append(val_accuracy)
        metrics["val_loss"].append(val_loss)

        print(f"Loss: {metrics['loss'][-1]:.4f}", end=' ')
        print(f"Accuracy: {metrics['accuracy'][-1]:.4f}", end=' ')
        print(f"Validation Loss: {metrics['val_loss'][-1]:.4f}", end=' ')
        print(f"Validation Accuracy: {metrics['val_accuracy'][-1]:.4f}")

        # Saving checkpoint
        if epoch >= warmup_epochs and val_loss < metrics["val_loss"][-2]:
            best_model = model
            torch.save({
                    "model_state": model.state_dict(),
                    "optimizer": optimizer.state_dict(),
                    "loss": criterion.state_dict(),
                    "epoch": epoch
            }, os.path.join(checkpoint, "state.pth"))

        # Early stopping
        if "early_stopping" in config:
            if val_loss < best_loss:
                best_loss_epoch = epoch
                best_loss = val_loss
            elif epoch - best_loss_epoch >= config["early_stopping"]["patience"]:
                print("Early stopping!")
                break

    # Calculate test metrics and confusion matrix
    test_accuracy, test_loss, test_targets, test_predictions = test_epoch(best_model,
                                                                          cinic_test, criterion)
    with open(os.path.join(checkpoint, "test_metrics.txt"), "w") as f:
        f.write(f"Test Loss: {test_loss:.4f} Test Accuracy: {test_accuracy:.4f}")

    # Confusion matrix
    disp = ConfusionMatrixDisplay.from_predictions(test_targets, test_predictions,
                                                   display_labels=classes)
    disp.figure_.savefig(os.path.join(checkpoint, "confusion_matrix_test.jpg"))

    # Confusion matrix without diagonal
    wrong_predictions_idx = test_targets != test_predictions
    test_targets = test_targets[wrong_predictions_idx]
    test_predictions = test_predictions[wrong_predictions_idx]
    disp = ConfusionMatrixDisplay.from_predictions(test_targets, test_predictions,
                                                   display_labels=classes)
    disp.figure_.savefig(os.path.join(checkpoint, "confusion_matrix_test_no_diag.jpg"))

    df = pd.DataFrame(metrics)
    df.to_csv(os.path.join(checkpoint, "metrics.csv"))


def train_epoch(
    model: nn.Module,
    train_ds: DataLoader,
    optimizer: torch.optim.Optimizer,
    criterion: nn.CrossEntropyLoss
):
    losses = []
    accuracies = []
    batch_sizes = []
    for input, target in tqdm(train_ds):
        optimizer.zero_grad()

        input, target = input.cuda(), target.cuda()
        ohe_target = F.one_hot(target, 10).type(torch.float32)
        output = model(input)
        loss = criterion(output, ohe_target)

        loss.backward()
        optimizer.step()

        pred = torch.argmax(output, dim=-1)
        accurate_pred = (pred == target).type(torch.float32)
        accuracies.append(torch.mean(accurate_pred).item())

        losses.append(loss.item())
        batch_sizes.append(len(input))

    accuracy = np.average(accuracies, weights=batch_sizes)
    loss = np.average(losses, weights=batch_sizes)
    return accuracy, loss


def valid_epoch(
    model: nn.Module,
    valid_ds: DataLoader,
    criterion: nn.CrossEntropyLoss
):
    val_losses = []
    val_accuracies = []
    batch_sizes = []
    with torch.no_grad():
        for input, target in tqdm(valid_ds):
            input, target = input.cuda(), target.cuda()
            ohe_target = F.one_hot(target, 10).type(torch.float32)
            output = model(input)
            val_loss = criterion(output, ohe_target)

            # Calculate accuracy
            pred = torch.argmax(output, dim=-1)
            accurate_pred = (pred == target).type(torch.float32)
            val_accuracies.append(torch.mean(accurate_pred).item())

            val_losses.append(val_loss.item())
            # Store information regarding
            batch_sizes.append(len(input))

    val_accuracy = np.average(val_accuracies, weights=batch_sizes)
    val_loss = np.average(val_losses, weights=batch_sizes)
    return val_accuracy, val_loss


def test_epoch(
    model: nn.Module,
    valid_ds: DataLoader,
    criterion: nn.CrossEntropyLoss
):
    val_losses = []
    val_accuracies = []
    batch_sizes = []
    predictions = []
    targets = []

    with torch.no_grad():
        for input, target in tqdm(valid_ds):
            input, target = input.cuda(), target.cuda()
            ohe_target = F.one_hot(target, 10).type(torch.float32)
            output = model(input)
            val_loss = criterion(output, ohe_target)

            # Calculate accuracy
            pred = torch.argmax(output, dim=-1)
            accurate_pred = (pred == target).type(torch.float32)
            val_accuracies.append(torch.mean(accurate_pred).item())

            val_losses.append(val_loss.item())
            # Store information regarding
            batch_sizes.append(len(input))

            predictions.append(pred)
            targets.append(target)

    val_accuracy = np.average(val_accuracies, weights=batch_sizes)
    val_loss = np.average(val_losses, weights=batch_sizes)
    predictions = torch.concatenate(predictions).cpu().numpy()
    targets = torch.concatenate(targets).cpu().numpy()
    return val_accuracy, val_loss, targets, predictions

In [5]:
cinic_train, cinic_val, cinic_test = get_cinic(DATA_PATH, 256)
checkpoint_path = os.path.join(CHECKPOINTS_PATH, "ResNet_lr_0_001")
state = torch.load(os.path.join(checkpoint_path, "state.pth"), weights_only=False)

In [6]:
state.keys()

dict_keys(['model_state', 'optimizer', 'loss', 'epoch'])

In [7]:
model = ResNet_32x32()
model.load_state_dict(state["model_state"])

<All keys matched successfully>

In [31]:
def evaluate(model: nn.Module, loader: DataLoader,):
    model = model.cuda()
    accuracies = []
    batch_sizes = []
    with torch.no_grad():
        for input, target in tqdm(loader):
            input, target = input.cuda(), target.cuda()
            output = model(input)
            predictions= torch.argmax(output, axis=-1)
            accuracies.append((torch.sum(predictions == target) / len(target)).reshape((-1)))
            batch_sizes.append(len(target))

    return accuracies, torch.mean(torch.concatenate(accuracies))

In [14]:
model.eval()
accuracies = evaluate(model, cinic_test)

  1%|          | 4/352 [00:08<09:44,  1.68s/it]

Accuracy:  0.8359375
Accuracy:  0.828125
Accuracy:  0.87109375
Accuracy:  0.84375
Accuracy:  0.8359375
Accuracy:  0.84375


  3%|▎         | 11/352 [00:09<02:20,  2.43it/s]

Accuracy:  0.78515625
Accuracy:  0.859375
Accuracy:  0.9296875
Accuracy:  0.89453125
Accuracy:  0.90625
Accuracy:  0.8125
Accuracy:  0.7578125


  5%|▌         | 19/352 [00:09<00:56,  5.89it/s]

...

 98%|█████████▊| 346/352 [00:22<00:00, 22.38it/s]

Accuracy:  0.6171875
Accuracy:  0.6484375
Accuracy:  0.52734375
Accuracy:  0.64453125
Accuracy:  0.71875
Accuracy:  0.81640625


 99%|█████████▉| 349/352 [00:22<00:00, 22.79it/s]

Accuracy:  0.828125
Accuracy:  0.80859375
Accuracy:  0.703125
Accuracy:  0.609375
Accuracy:  0.625


100%|██████████| 352/352 [00:23<00:00, 14.96it/s]


In [ ]:
torch.mean(torch.concatenate([acc.reshape((-1)) for acc in accuracies]))

tensor(0.7342, device='cuda:0')

In [37]:
model.eval()
accuracies_train, acc_train = evaluate(model, cinic_train)
acc_train

100%|██████████| 352/352 [03:10<00:00,  1.85it/s]


tensor(0.8439, device='cuda:0')

In [33]:
accuracies_val, acc_val = evaluate(model, cinic_val)
acc_val

100%|██████████| 352/352 [02:56<00:00,  2.00it/s]


tensor(0.7361, device='cuda:0')